In [1]:
from tinygrad import Tensor, nn


In [4]:
t = Tensor([1, 2, 3, 4])
print(t.numpy())
print(t.unsqueeze(0).numpy())

[1 2 3 4]
[[1 2 3 4]]


In [5]:
print(t.unsqueeze(1).numpy())


[[1]
 [2]
 [3]
 [4]]


In [6]:
print(t.unsqueeze(0).unsqueeze(1).numpy())

[[[1 2 3 4]]]


In [7]:
print(t.unsqueeze(0).unsqueeze(1).numpy().shape)

(1, 1, 4)


In [8]:
print(t.unsqueeze(0,1).numpy().shape)

TypeError: Tensor.unsqueeze() takes 2 positional arguments but 3 were given

In [17]:
print(t.unsqueeze(1).unsqueeze(1).squeeze(1).numpy().shape)

(4, 1)


In [22]:
a = Tensor.arange(6).reshape(2,3)
print(a.numpy())

[[0 1 2]
 [3 4 5]]


In [23]:
b = a.reshape(3,2)
print(b.numpy())

[[0 1]
 [2 3]
 [4 5]]


In [30]:
import math

def pretty_str(data, shape, indent=0):
    """
    Recursively build a string representation of 'data' given 'shape'.
    'data' is a 1D list (or similar) storing the tensor's elements.
    'shape' is a tuple/list indicating the dimensions of the tensor.
    'indent' is used internally to control indentation for deeper dimensions.
    """
    # Base case: 1D, just print elements in brackets.
    if len(shape) == 1:
        # Extract exactly 'shape[0]' items from 'data' for this slice
        row_data = data[:shape[0]]
        # Format the row with a space (or comma) separator
        return "[" + " ".join(str(x) for x in row_data) + "]"
    
    # Recursive case: N > 1
    # We'll slice out 'subslice_size' elements for each index in the first dimension.
    subslice_size = math.prod(shape[1:])  # Number of elements in each sub-tensor
    print((shape,shape[1:],subslice_size))
    lines = []
    for i in range(shape[0]):
        # For dimension i, extract the corresponding chunk from data
        start = i * subslice_size
        end = (i + 1) * subslice_size
        sub_data = data[start:end]
        
        # Recursively build the string for this sub-tensor
        sub_str = pretty_str(sub_data, shape[1:], indent + 1)
        lines.append(sub_str)
    
    # Join the sub-tensors with newlines (and some indentation if desired)
    # You could also choose to put them on one line, etc.
    indent_str = "  " * indent
    inner = (",\n" + indent_str).join(lines)
    return "[" + "\n" + indent_str + inner + "\n" + indent_str + "]"
# Suppose you have data in a linear list:
data = list(range(2*3*4))  # e.g., [0,1,2,3,4,5, ... 23]
shape = (2, 3, 4)         # a 2x3x4 tensor

s = pretty_str(data, shape)
print("tensor(" + s + f", shape={shape})")

((2, 3, 4), (3, 4), 12)
((3, 4), (4,), 4)
((3, 4), (4,), 4)
tensor([
[
  [0 1 2 3],
  [4 5 6 7],
  [8 9 10 11]
  ],
[
  [12 13 14 15],
  [16 17 18 19],
  [20 21 22 23]
  ]
], shape=(2, 3, 4))


tensor([
[
  [0 1 2 3],
  [4 5 6 7],
  [8 9 10 11]
  ],
[
  [12 13 14 15],
  [16 17 18 19],
  [20 21 22 23]
  ]
], shape=(2, 3, 4))


In [26]:
import math
import functools

def _print_recursive(data, shape, dim=0, index=0):
    """
    Recursively construct a string representation of the data with given shape.

    data: 1D list containing the actual values
    shape: tuple or list of dimensions (e.g. (2,3,4))
    dim: current dimension we are printing
    index: current index in 'data' where this sub-tensor begins
    """
    # Base case: if we are at the last dimension, just print the elements
    if dim == len(shape) - 1:
        size = shape[dim]
        row = data[index : index + size]
        return "[" + ", ".join(str(x) for x in row) + "]"
    
    # Otherwise, we need to recurse for each “slice” along the current dim
    size = shape[dim]
    sub_strs = []
    sub_shape_size = functools.reduce(lambda a, b: a*b, shape[dim+1:], 1)

    # For each slice along the current dimension, descend one level
    for i in range(size):
        sub_index = index + i * sub_shape_size
        sub_strs.append(_print_recursive(data, shape, dim+1, sub_index))

    # Join the sub-slices with commas + possible newlines
    # Add extra newlines or spaces if you want a more “pretty” multiline format
    inside = ",\n ".join(sub_strs)
    return "[" + inside + "]"


class MyTensor:
    def __init__(self, data, shape):
        """
        data: a 1D list or array storing values in row-major order
        shape: tuple or list describing the dimensions (e.g., (2, 3, 4))
        """
        self.data = data
        self.shape = shape
    
    def __repr__(self):
        # Build a string that looks like PyTorch: "tensor([...])"
        # Or for numpy style, you could do "array([...])"
        return "tensor(" + _print_recursive(self.data, self.shape) + ")"

# Suppose shape is (2, 2, 3), so we have 12 elements in data
data = list(range(1, 13))  # [1, 2, 3, ..., 12]
shape = (2, 2, 3)
t = MyTensor(data, shape)
print(t)


tensor([[[1, 2, 3],
 [4, 5, 6]],
 [[7, 8, 9],
 [10, 11, 12]]])
